In [2]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64


#LOAD AND PREPROCESS THE DATA
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
17473536/17464789 [==============================] - 0s 0us/step


In [3]:
#CREATING THE MODEL
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [4]:
#TRAIN

model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

625/625 [==============================] - 15s 12ms/step - loss: 0.4156 - acc: 0.8122 - val_loss: 0.2951 - val_acc: 0.8792
Epoch 2/10
625/625 [==============================] - 8s 12ms/step - loss: 0.2371 - acc: 0.9103 - val_loss: 0.2728 - val_acc: 0.8886
Epoch 3/10
625/625 [==============================] - 7s 11ms/step - loss: 0.1817 - acc: 0.9328 - val_loss: 0.2915 - val_acc: 0.8916
Epoch 4/10
625/625 [==============================] - 7s 11ms/step - loss: 0.1498 - acc: 0.9466 - val_loss: 0.3100 - val_acc: 0.8826
Epoch 5/10
625/625 [==============================] - 7s 12ms/step - loss: 0.1278 - acc: 0.9561 - val_loss: 0.3538 - val_acc: 0.8794
Epoch 6/10
625/625 [==============================] - 7s 11ms/step - loss: 0.1110 - acc: 0.9633 - val_loss: 0.2856 - val_acc: 0.8854
Epoch 7/10
625/625 [==============================] - 7s 12ms/step - loss: 0.0939 - acc: 0.9682 - val_loss: 0.3166 - val_acc: 0.8782
Epoch 8/10
625/625 [==============================] - 8s 12ms/step - loss: 0.08

In [5]:
#CHECK RESULTS

results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 4s 6ms/step - loss: 0.4948 - acc: 0.8493
[0.4947705864906311, 0.8493199944496155]


In [9]:
#MAKE PREDICTIONS

word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  print(tokens)
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)

[12, 17, 13, 40, 477, 35, 477]


In [7]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))
print(reverse_word_index )

that movie was just amazing so amazing
{34701: 'fawn', 52006: 'tsukino', 52007: 'nunnery', 16816: 'sonja', 63951: 'vani', 1408: 'woods', 16115: 'spiders', 2345: 'hanging', 2289: 'woody', 52008: 'trawling', 52009: "hold's", 11307: 'comically', 40830: 'localized', 30568: 'disobeying', 52010: "'royale", 40831: "harpo's", 52011: 'canet', 19313: 'aileen', 52012: 'acurately', 52013: "diplomat's", 25242: 'rickman', 6746: 'arranged', 52014: 'rumbustious', 52015: 'familiarness', 52016: "spider'", 68804: 'hahahah', 52017: "wood'", 40833: 'transvestism', 34702: "hangin'", 2338: 'bringing', 40834: 'seamier', 34703: 'wooded', 52018: 'bravora', 16817: 'grueling', 1636: 'wooden', 16818: 'wednesday', 52019: "'prix", 34704: 'altagracia', 52020: 'circuitry', 11585: 'crotch', 57766: 'busybody', 52021: "tart'n'tangy", 14129: 'burgade', 52023: 'thrace', 11038: "tom's", 52025: 'snuggles', 29114: 'francesco', 52027: 'complainers', 52125: 'templarios', 40835: '272', 52028: '273', 52130: 'zaniacs', 34706: '275

In [12]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was really  it and would   it  because it was "
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


[12, 17, 13, 63, 9, 2, 59, 103, 9, 171, 85, 9, 13]
[0.75487655]
[12, 17, 63, 2064, 10, 1797, 9, 2, 583, 103, 9, 171, 13, 28, 4, 1, 246, 180, 204, 123, 293]
[0.19642775]
